# Data

In [1]:
%load_ext autoreload
%autoreload 2
%cd /home/mi/au/spbmetro

from ipyannotate import annotate
from ipyannotate.buttons import (ValueButton as Button, BackButton, NextButton)

def check(rule, tree=False, facts=False):
    return annotate(texts, buttons=[
        Button('ok', color='green', shortcut='1'),
        Button('err', color='red', shortcut='2'),
        BackButton(shortcut='j'), NextButton(shortcut='k'),
    ], display=lambda text: test(rule, text, tree=tree, facts=facts))

from Levenshtein import distance
from yargy.tokenizer import MorphTokenizer
from yargy.predicates import gram, in_, type, normalized, eq, custom
from yargy.pipelines import morph_pipeline
from yargy.relations import gnc_relation, main
from yargy import or_, and_, not_, rule, Parser
from yargy import interpretation as interp
from yargy.interpretation import fact, attribute
from yargy import interpretation as interp
import yargy
import re

from notebook.common import *
from notebook.helper import test, test_samples


from structure import History, Subway
texts = [_.text for _ in History.load().messages.values()]

/home/mi/work/spbmetro


# Grammar

In [2]:
[t for t in TOKENIZER(' работает только на выход ')]

[MorphToken('работает',
            [1, 9),
            'RU',
            [Form('работать', Grams(3per,VERB,impf,indc,intr,pres,sing))]),
 MorphToken('только',
            [10, 16),
            'RU',
            [Form('только', Grams(PRCL)), Form('только', Grams(CONJ))]),
 MorphToken('на',
            [17, 19),
            'RU',
            [Form('на', Grams(PREP)),
             Form('на', Grams(PRCL)),
             Form('на', Grams(INTJ))]),
 MorphToken('выход',
            [20, 25),
            'RU',
            [Form('выход', Grams(NOUN,accs,inan,masc,sing)),
             Form('выход', Grams(NOUN,inan,masc,nomn,sing))])]

## Time

In [3]:
from notebook.time import TIME

test(TIME,
     ['в  12:31 но не в 33:33 и не в 09:90',               'в  12:31'],
     ['что с  1:09  по  8:09  с  8:20  до  9:00  ч вход',  '1:09', '8:09', '8:20', '9:00  ч'],
     ['что  10:02 .2018 и 10.02.2018 станция закрыта'],
     ['с16ч 04 мин до конца рабочего дня станция закрыта', '16ч 04 мин'],
     ['в 17 ч02 мин александра невского 1-2 закрыт в  18.40 и в 18- 40 ', 
                                                           'в 17 ч02 мин', 'в  18.40', 'в 18- 40'],
     ['баллончик в  18:10 ч. нарушитель, с 10-27 мин',     'в  18:10 ч.', '10-27 мин'],
     ['пункт 2. 16-40 закрыта платформа',                  '16-40'],
     facts=False, tree=False
     )

LineMarkup('в  12:31 но не в 33:33 и не в 09:90', [Span(0, 8, '#aec7e8')])

LineMarkup('что с  1:09  по  8:09  с  8:20  до  9:00  ч вход',
           [Span(7, 11, '#aec7e8'),
            Span(17, 21, '#aec7e8'),
            Span(26, 30, '#aec7e8'),
            Span(36, 43, '#aec7e8')])

LineMarkup('что  10:02 .2018 и 10.02.2018 станция закрыта',
           [Span(5, 10, '#ff9896'), Span(19, 24, '#ff9896')])

LineMarkup('с16ч 04 мин до конца рабочего дня станция закрыта',
           [Span(1, 11, '#aec7e8')])

LineMarkup('в 17 ч02 мин александра невского 1-2 закрыт в  18.40 и в 18- 40 ',
           [Span(0, 12, '#aec7e8'),
            Span(44, 52, '#aec7e8'),
            Span(55, 63, '#aec7e8')])

LineMarkup('баллончик в  18:10 ч. нарушитель, с 10-27 мин',
           [Span(10, 21, '#aec7e8'), Span(36, 45, '#aec7e8')])

LineMarkup('пункт 2. 16-40 закрыта платформа',
           [Span(6, 11, '#ff9896'), Span(9, 14, '#ccc')])

In [4]:
# annotate_time = check(TIME, facts=True);
# annotate_time
# print('failed:', len([_ for _ in annotate_time.tasks if _.value != 'ok']), '/', len(annotate_time.tasks))

## Duration

In [5]:
from notebook.duration import DURATION

test(DURATION,
     ['с  12:31 но не с 33:33 до 09:90 а до 09:30', 'с  12:31', 'до 09:30'],
     ['что с  1:09  по  8:09  с  8:20  до  9:00  ч вход', 'с  1:09  по  8:09', 'с  8:20  до  9:00  ч'],
     ['что с 10:02 .2018 и с 10.02.2018 станция закрыта'],
     ['с16ч 04 мин до конца рабочего дня станция закрыта', 'с16ч 04 мин до конца рабочего дня'],
     ['с 17 ч02 мин александра невского 1-2 до конечной станции закрыты с 18:40 и с 18- 40',
          'с 17 ч02 мин', 'с 18:40', 'с 18- 40'],
     ['с  18:10 ч. закрыта, с 10-27 мин тоже', 'с  18:10 ч.', 'с 10-27 мин'],
     ['с 17-00 до окончания будет', 'с 17-00 до окончания'],
     facts=False, tree=False
     )

LineMarkup('с  12:31 но не с 33:33 до 09:90 а до 09:30',
           [Span(0, 8, '#aec7e8'), Span(34, 42, '#aec7e8')])

LineMarkup('что с  1:09  по  8:09  с  8:20  до  9:00  ч вход',
           [Span(4, 21, '#aec7e8'), Span(23, 43, '#aec7e8')])

LineMarkup('что с 10:02 .2018 и с 10.02.2018 станция закрыта',
           [Span(4, 11, '#ff9896'), Span(20, 27, '#ff9896')])

LineMarkup('с16ч 04 мин до конца рабочего дня станция закрыта',
           [Span(0, 33, '#aec7e8')])

LineMarkup('с 17 ч02 мин александра невского 1-2 до конечной станции закрыты с 18:40 и с 18- 40',
           [Span(0, 12, '#aec7e8'),
            Span(65, 72, '#aec7e8'),
            Span(75, 83, '#aec7e8')])

LineMarkup('с  18:10 ч. закрыта, с 10-27 мин тоже',
           [Span(0, 11, '#aec7e8'), Span(21, 32, '#aec7e8')])

LineMarkup('с 17-00 до окончания будет', [Span(0, 20, '#aec7e8')])

## Literals

In [22]:
from notebook.literal import *

test(LITERAL, ['без двух; один, два; первый, второй', 'двух', 'один', 'два', 'первый', 'второй'], facts=False)
test(LIST_OF_NUMERALS, ['восстания 1, вестибюль - 1, 2;   невского 1 и 2', '1, 2', '1 и 2', '1'], facts=False)
test(LIST_OF_LITERALS, ['второй и первый павильоны; первый - второй вестибюли', 'второй и первый', 'первый - второй'], facts=False)

LineMarkup('без двух; один, два; первый, второй',
           [Span(4, 8, '#aec7e8'),
            Span(10, 14, '#aec7e8'),
            Span(16, 19, '#aec7e8'),
            Span(21, 27, '#aec7e8'),
            Span(29, 35, '#aec7e8')])

LineMarkup('восстания 1, вестибюль - 1, 2;   невского 1 и 2',
           [Span(10, 11, '#aec7e8'),
            Span(25, 29, '#aec7e8'),
            Span(42, 47, '#aec7e8')])

LineMarkup('второй и первый павильоны; первый - второй вестибюли',
           [Span(0, 15, '#aec7e8'), Span(27, 42, '#aec7e8')])

## Vestibules

In [7]:
from notebook.vestibule import VESTIBULE

test(VESTIBULE,
    ['вестибюли 1 и 2 ст . ленинский проспект открыт', 'вестибюли 1 и 2'],
    ['Временно открыт вестибюль 2 ст. Достоевская на вход и выход.', 'вестибюль 2'],
    ['Второй и первый вестибюли открыты', 'Второй и первый вестибюли'],
    ['вестибюль №2 ст. пр. Ветеранов закрыт'],
    facts=True)

LineMarkup('вестибюли 1 и 2 ст . ленинский проспект открыт',
           [Span(0, 15, '#aec7e8')])

Vestibules(num=[1, 2])

LineMarkup('Временно открыт вестибюль 2 ст. Достоевская на вход и выход.',
           [Span(16, 27, '#aec7e8')])

Vestibules(num=[2])

LineMarkup('Второй и первый вестибюли открыты', [Span(0, 25, '#aec7e8')])

Vestibules(num=[1, 2])

LineMarkup('вестибюль №2 ст. пр. Ветеранов закрыт', [Span(0, 9, '#ff9896')])

Vestibules(num=None)

## Station titles

In [8]:
from notebook.station_title import STATION_TITLE

test(STATION_TITLE,
     ['Гостинку и гостиный двор 28 апреля 2018 года ст . приморская закрыта', 'гостиный двор', 'Гостинку', 'приморская'],
     ['техн и-т закрыт; переход тн 1 открыт, для тех кто не знал ', 'техн и-т', 'тн'],
     [' ст . технологический ин - т открыта . опасных предметов не обнаружено', 'технологический ин - т'],
     ['выход на улицу восстания и московский вокзал; спортивная сумка', ],
     ['В 19-54 ст. Ст. Деревня открыта, опасных предметов не обнаружено.', 'Ст. Деревня'],
     ['в 19 - 54 ст . . деревня открыта , ст. Ст. Гражданский пр.', 'ст . . деревня', 'Гражданский пр.'],
     ['ст . технологический - 1 и переход на ст . технолог . - 2 ', 'технологический', 'технолог .'],
     ['комментарии губернатора спб . полтавченко г . с '],
     ['ст . гостинный двор; на ст . ладожкская снято', 'гостинный двор', 'ладожкская'],
     facts=False, tree=False)

LineMarkup('Гостинку и гостиный двор 28 апреля 2018 года ст . приморская закрыта',
           [Span(0, 8, '#aec7e8'),
            Span(11, 24, '#aec7e8'),
            Span(50, 60, '#aec7e8')])

LineMarkup('техн и-т закрыт; переход тн 1 открыт, для тех кто не знал ',
           [Span(0, 8, '#aec7e8'), Span(25, 27, '#aec7e8')])

LineMarkup(' ст . технологический ин - т открыта . опасных предметов не обнаружено',
           [Span(6, 28, '#aec7e8')])

LineMarkup('выход на улицу восстания и московский вокзал; спортивная сумка',
           [Span(15, 24, '#ff9896'),
            Span(27, 37, '#ff9896'),
            Span(46, 56, '#ff9896')])

LineMarkup('В 19-54 ст. Ст. Деревня открыта, опасных предметов не обнаружено.',
           [Span(12, 23, '#aec7e8')])

LineMarkup('в 19 - 54 ст . . деревня открыта , ст. Ст. Гражданский пр.',
           [Span(10, 24, '#ccc'), Span(43, 58, '#aec7e8')])

LineMarkup('ст . технологический - 1 и переход на ст . технолог . - 2 ',
           [Span(5, 20, '#aec7e8'), Span(43, 53, '#aec7e8')])

LineMarkup('комментарии губернатора спб . полтавченко г . с ',
           [Span(42, 45, '#ff9896')])

LineMarkup('ст . гостинный двор; на ст . ладожкская снято',
           [Span(5, 19, '#ccc'), Span(29, 39, '#ccc')])

## Stations & Transfers

In [9]:
from notebook.transfer import TRANSFER

test(TRANSFER,
     ['переход Гостиный двор- Невский пр. открыт', 'переход Гостиный двор- Невский пр.'],
     ['Переход со ст. м. «Озерки» на ст.Владимирская закрыт', 'Переход со ст. м. «Озерки» на ст.Владимирская'],
     ['переход на верхнюю платформу закрыт', 'переход на верхнюю платформу'],
     ['Ст. пл. Ал.Невского2 и переход Ал.Невского1-2 закрыт', 'переход Ал.Невского1-2'],
     ['и переходы с Гостиного двора открыты', 'переходы с Гостиного двора'],
     ['переход Влад.- Достоевская закрыты', 'переход Влад.- Достоевская'],
     ['переходы на Сенную пл. и Спасскую закрыты', 'переходы на Сенную пл. и Спасскую'],
     ['переход Тн1- Тн2 открыт', 'переход Тн1- Тн2'],
     ['переход между ст. Сенная площадь и Спасская закрыт', 'переход между ст. Сенная площадь и Спасская'],
     ['переход между ст. Технологический институт 1 и 2 открыт', 'переход между ст. Технологический институт 1 и 2'],
     ['закрыт переход Достоевская -Владимирская', 'переход Достоевская -Владимирская'],
    facts=False)

# test_samples([LIST_OF_STATIONS.named('2')], texts, fact=True)

LineMarkup('переход Гостиный двор- Невский пр. открыт',
           [Span(0, 34, '#aec7e8')])

LineMarkup('Переход со ст. м. «Озерки» на ст.Владимирская закрыт',
           [Span(0, 45, '#aec7e8')])

LineMarkup('переход на верхнюю платформу закрыт',
           [Span(0, 7, '#ff9896'), Span(0, 28, '#ccc')])

LineMarkup('Ст. пл. Ал.Невского2 и переход Ал.Невского1-2 закрыт',
           [Span(23, 45, '#aec7e8')])

LineMarkup('и переходы с Гостиного двора открыты', [Span(2, 28, '#aec7e8')])

...

LineMarkup('переходы на Сенную пл. и Спасскую закрыты',
           [Span(0, 33, '#aec7e8')])

LineMarkup('переход Тн1- Тн2 открыт', [Span(0, 16, '#aec7e8')])

LineMarkup('переход между ст. Сенная площадь и Спасская закрыт',
           [Span(0, 43, '#aec7e8')])

LineMarkup('переход между ст. Технологический институт 1 и 2 открыт',
           [Span(0, 48, '#aec7e8')])

LineMarkup('закрыт переход Достоевская -Владимирская',
           [Span(7, 40, '#aec7e8')])

In [10]:
from notebook.transfer import STATION_AND_TRANSFER, TRANSFER, STATION

test(STATION_AND_TRANSFER,
    ['в 14 - 09 ст . спасская и переходы на ст . садовая и ст . сенная площадь закрыты из - за бесхозного предмета', 'ст . спасская и переходы на ст . садовая и ст . сенная площадь'],
    ['ст . сенная площадь и переходы на ст . садовая и ст . спасская закрыты из - за ананомного сообщения', 'ст . сенная площадь и переходы на ст . садовая и ст . спасская'],
    ['Ст. «пл. Ал.Невского2» и переход Ал.Невского1-2 закрыт', 'Ст. «пл. Ал.Невского2» и переход Ал.Невского1-2'],
    facts=False)

# test_samples([STATION_AND_TRANSFER.named(''), TRANSFER.named('1')], texts, markup=LineMarkup)

LineMarkup('в 14 - 09 ст . спасская и переходы на ст . садовая и ст . сенная площадь закрыты из - за бесхозного предмета',
           [Span(10, 72, '#aec7e8')])

LineMarkup('ст . сенная площадь и переходы на ст . садовая и ст . спасская закрыты из - за ананомного сообщения',
           [Span(0, 62, '#aec7e8')])

LineMarkup('Ст. «пл. Ал.Невского2» и переход Ал.Невского1-2 закрыт',
           [Span(0, 47, '#aec7e8')])

In [11]:
from notebook.working import *

test(WORKING,
    ['переход на верхнюю платформу закрыт', 'закрыт'],
    ['Станция была на вход закрыта. станция не работает.', 'была на вход закрыта', 'не работает'],
    ['Работает и на вход и на выход', 'Работает и на вход и на выход'],
    ['Девяткино закрыта на вход (на выход работает)', 'закрыта на вход', 'на выход работает'],
    ['институт закрыта на вход, на выход работает', 'закрыта на вход', 'на выход работает'],
    ['Чернышевская на выход работает, закрыта только на вход, работает только выход',
     'на выход работает', 'закрыта только на вход', 'работает только выход'],
    ['Павильон-2 работает в нормальном режиме', 'работает в нормальном режиме'],
    ['Девяткино не закрыта на вход (выход не работал)', 'не закрыта на вход', 'выход не работал'],
    facts=False)

LineMarkup('переход на верхнюю платформу закрыт', [Span(29, 35, '#aec7e8')])

LineMarkup('Станция была на вход закрыта. станция не работает.',
           [Span(8, 28, '#aec7e8'), Span(38, 49, '#aec7e8')])

LineMarkup('Работает и на вход и на выход', [Span(0, 29, '#aec7e8')])

LineMarkup('Девяткино закрыта на вход (на выход работает)',
           [Span(10, 25, '#aec7e8'), Span(27, 44, '#aec7e8')])

LineMarkup('институт закрыта на вход, на выход работает',
           [Span(9, 24, '#aec7e8'), Span(26, 43, '#aec7e8')])

LineMarkup('Чернышевская на выход работает, закрыта только на вход, работает только выход',
           [Span(13, 30, '#aec7e8'),
            Span(32, 54, '#aec7e8'),
            Span(56, 77, '#aec7e8')])

LineMarkup('Павильон-2 работает в нормальном режиме',
           [Span(11, 39, '#aec7e8')])

LineMarkup('Девяткино не закрыта на вход (выход не работал)',
           [Span(10, 28, '#aec7e8'), Span(30, 46, '#aec7e8')])

In [36]:
test_samples([TIME.named('1'), STATION_AND_TRANSFER.named(''), WORKING.named('2'), VESTIBULE.named('3')], texts, fact=False)

BoxLabelMarkup('В 17-22 ст. Невский Пр, Гостиный Двор, переходы закрыты в связи с осмотром станции.',
               [Span(0, 7, '1'),
                Span(8, 22, ''),
                Span(24, 47, ''),
                Span(48, 55, '2')])

BoxLabelMarkup('В 16-54 ст. Маяковская и переход на ст. Площадь Восстания закрыты из-за бесхозного предмета',
               [Span(0, 7, '1'), Span(8, 57, ''), Span(58, 65, '2')])

BoxLabelMarkup('В 18:35 ст. Старая деревня открыта. Опасных предметов не обнаружено',
               [Span(0, 7, '1'), Span(8, 26, ''), Span(27, 34, '2')])

BoxLabelMarkup('В 13-20 ст. Ладожская закрыта, проводится проверка бесхозного предмета.',
               [Span(0, 7, '1'), Span(8, 21, ''), Span(22, 29, '2')])

BoxLabelMarkup('В 15-26 ст. Спортивная закрыта из-за бесхозного предмета',
               [Span(0, 7, '1'), Span(8, 22, ''), Span(23, 30, '2')])

...

BoxLabelMarkup('В 11-20 ст. Достоевская закрыта из-за бесхозного предмета',
               [Span(0, 7, '1'), Span(8, 23, ''), Span(24, 31, '2')])

BoxLabelMarkup('В 13-43 вестибюль 1 ст. Проспект Ветеранов закрыт из-за бесхозного предмета',
               [Span(0, 7, '1'),
                Span(8, 19, '3'),
                Span(20, 42, ''),
                Span(43, 49, '2')])

BoxLabelMarkup('В 14-46 ст. Горьковская открыта. Опасных предметов не обнаружено.',
               [Span(0, 7, '1'), Span(8, 23, ''), Span(24, 31, '2')])

BoxLabelMarkup('В 21-43 ст. Гражданский проспект открыта. Опасных предметов не обнаружено',
               [Span(0, 7, '1'), Span(8, 32, ''), Span(33, 40, '2')])

BoxLabelMarkup('В 20-19 ст.Фрунзенская открыта, опасных предметов не обнаружено.',
               [Span(0, 7, '1'), Span(8, 22, ''), Span(23, 30, '2')])

### Интересные примеры

1. Здесь нужно натравливать несколько правил (transfer, station&transfer, итд), и выкидывать из них те, что "включены" остальными правилами. (см. пример №3)
2. Нужно делать вычисления над булевыми формулами. Например, в примере №6 нужно взять конъюнкцию "только вход" и "работает всё", это поможет избежать ошибок. 
3. Возможно нужно использовать локальность фраз: например, брать первые три типа фраз слева-направо. 

In [34]:
test_samples([TIME.named('1'), STATION_AND_TRANSFER.named(''), WORKING.named('2'), VESTIBULE.named('3')], [
    'В 14-08 выход на ст. Московская-1 открыт.',
    'В 11-30 вход на ст. Приморская возобновлен',
    'В 22-15 закрыт переход Г.Двор- Невский пр. На переходе бесхозный пакет.',
    'В 10-28 вестибюль №2 ст. пр. Ветеранов закрыт',  # не подземная станция
    'В 16-23 ст. Купчино, в 16-35 ст.Просвещения открыты',
    'В 16-30 ст.Крестовский остров закрыта на вход, работают все 3 эскалатора на выхода',
    'В 13-34 вход на ст. Кировский завод ограничен',
    'В 18-40 ст. Новокрестовская павильон-1 временно закрыт из-за ветра. Павильон-2 работает в обычном режиме',
], fact=False)

BoxLabelMarkup('В 14-08 выход на ст. Московская-1 открыт.',
               [Span(0, 7, '1'), Span(17, 33, ''), Span(34, 40, '2')])

BoxLabelMarkup('В 11-30 вход на ст. Приморская возобновлен',
               [Span(0, 7, '1'), Span(16, 30, '')])

BoxLabelMarkup('В 22-15 закрыт переход Г.Двор- Невский пр. На переходе бесхозный пакет.',
               [Span(0, 7, '1'),
                Span(8, 14, '2'),
                Span(23, 29, ''),
                Span(31, 42, '')])

BoxLabelMarkup('В 10-28 вестибюль №2 ст. пр. Ветеранов закрыт',
               [Span(0, 7, '1'),
                Span(8, 17, '3'),
                Span(21, 38, ''),
                Span(39, 45, '2')])

BoxLabelMarkup('В 16-23 ст. Купчино, в 16-35 ст.Просвещения открыты',
               [Span(0, 7, '1'),
                Span(8, 19, ''),
                Span(21, 28, '1'),
                Span(29, 43, ''),
                Span(44, 51, '2')])

BoxLabelMarkup('В 16-30 ст.Крестовский остров закрыта на вход, работают все 3 эскалатора на выхода',
               [Span(0, 7, '1'),
                Span(8, 29, ''),
                Span(30, 45, '2'),
                Span(47, 55, '2')])

BoxLabelMarkup('В 13-34 вход на ст. Кировский завод ограничен',
               [Span(0, 7, '1'), Span(16, 35, '')])

BoxLabelMarkup('В 18-40 ст. Новокрестовская павильон-1 временно закрыт из-за ветра. Павильон-2 работает в обычном режиме',
               [Span(0, 7, '1'),
                Span(8, 27, ''),
                Span(28, 38, '3'),
                Span(48, 54, '2'),
                Span(68, 78, '3'),
                Span(79, 104, '2')])

In [14]:
import pandas as pd
from datetime import datetime as dt
%run structure.py

x = [_.date for _ in History.load()]
d = pd.DataFrame({"date": pd.to_datetime(x, unit='s')})
d['day'] = d['date'].map(lambda d: d.toordinal())

kek = d.day.value_counts()
kek
e = kek.reset_index(name='freq').set_index(kek.index.map(dt.fromordinal))
e.freq = e.freq.map(lambda x: x // 2)
# (e / 734).freq.sum()
# e